# Balanced Search Trees

### Search Trees

* `find(), insert()` and `delete()` all walk down a single path
* Worst-case: height of the tree
* An un-balanced tree with $n$ nodes may have height $O(n)$
* Balanced trees have height $O(log \ n)$
* How can we maintain balance as the tree grows and shrinks

**Defining balance**
* Left and Right sub-trees should be "equal"
  - Two possible measures: `size` (number of nodes) and `height`
* `self.left.size()` and `self.right.size()` are equal?
  - Only possible for **`complete`** binary trees!
* `self.left.size()` and `self.right.size()` differ by at most 1?
  - Plausible, but rather difficult to maintain!

### Height Balanced Trees

* `self.height()` -- number of nodes on the longest path from root to leaf
  - $0$ for an empty tree
  - $1$ for tree with only a root node
  - $1 + max$ of heights of left and right sub-trees, in general
* Height balance
  - `self.left.height()` and `self.right.height()` differ by at most $1$
  - AVL trees -- Adelson-Velskii, Landis
* Does height balance guarantee $O(log \ n)$ height?
* Minimum size height-balanced trees

![Tree](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/W7L1_1.png?alt=media&token=bdd6ee5d-82a6-4255-8801-cd6fbce7ffad)

* General strategy to build a small balanced tree of height `h`
  - Smallest balanced tree of height $h - 1$ as the left sub-tree
  - Smallest balanced tree of height $h - 2$ as the right sub-tree

-------------------------------------------------------------------------------
* $S(h)$, is the size of the smallest height-balanced tree of height $h$
* Recurrence
  - $S(0) = 0, S(1) = 1$
  - $S(h) = 1 + S(h - 1) + S(h - 2)$
* Compare to Fibonacci sequence
  - $F(0) = 0, F(1) = 1$
  - $F(n) = F(n - 1) + F(n - 2)$
* $S(h)$ grows exponentially with $h$ $\Leftrightarrow$ For size $n$, $h$ is $O(log \ n)$

### Correcting Imbalance

* Slope of a node: `self.left.height() - self.right.height()`
* Balanced tree -- slope is $\{-1, 0, 1\}$
* Operation such as `tree.insert(value), tree.delete(value)` can alter the slope to $-2$ or $+2$

**Left Rotation** -- converts slope $-2$ to $\{0, 1, 2\}$

![Tree](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/W7L1_2.png?alt=media&token=d1e156c7-7a31-4eae-9560-412059ecec7b)

**Right Rotations** -- converts slope $+2$ to $\{-2, -1, 0\}$

![Tree](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/W7L1_3.png?alt=media&token=dfc707da-ec64-4567-837b-1ebe8f440715)

### Implementing rotations (Left rotate)

![Tree](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/W7L1_4.png?alt=media&token=d22281d4-f87e-4ef8-bdd0-ff65999d941e)

In [ ]:
class Tree:
  ...
  def left_rotate(self):
    v = self.value
    vr = self.right.value
    tl = self.left
    trl = self.right.left
    trr = self.right.right

    new_left = Tree(v)
    new_left.left = tl
    new_left.right = trl

    self.value = vr
    self.right = trr
    self.left = new_left

    return

### Implementing Rotations (Right rotate)

![Tree](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/W7L1_5.png?alt=media&token=828385e2-9f58-4c36-945d-f6a2888f2c66)

In [ ]:
class Tree:
  ...
  def right_rotate(self):
    v = self.value
    vl = self.left.value
    tll = self.left.left
    tlr = self.left.right
    tr = self.right

    new_right = Tree(v)
    new_right.left = tlr
    new_right.right = tr

    self.value = vl
    self.left = tll
    self.right = new_right

    return

### Update `insert()` and `delete()`

* Use the re-balancing strategy to define a function `rebalance()`
* Re-balance each time the tree is modified
* Automatically re-balances bottom up

In [ ]:
class Tree:
  ...
  def insert(self, v):
    if self.is_empty():
      self.value = v
      self.left = Tree()
      self.right = Tree()
    
    if self.value == v:
      return
    
    if v < self.value:
      self.left.insert(v)
      self.left.re_balance()
      return
    
    if v > self.value:
      self.right.insert(v)
      self.right.re_balance()
      return

In [ ]:
class Tree:
  ...
  def delete(self, v):
    ...
    if v < self.value:
      self.left.delete(v)
      self.left.re_balance()
      return
    
    if v > self.value:
      self.right.delete(v)
      self.right.re_balance()
      return
    
    if v == self.value:
      if self.is_leaf():
        self.make_empty()
      elif self.left.is_empty():
        self.copy_right()
      elif self.right.is_empty():
        self.copy_left()
      else:
        self.value = self.left.max_val()
        self.left.delete(self.left.max_val())
      return

### Computing slope

* To compute the slope we need the heights of the sub-trees
* But, computing the height is $O(n)$

In [ ]:
class Tree:
  ...
  def height(self):
    if self.is_empty():
      return 0
    else:
      return 1 + max(self.left.height(), self.right.height())

* Instead, maintain a field `self.height`
* After each modification, update `self.height` based on `self.left.height, self.right.height`

In [ ]:
class Tree:
  ...
  def insert(self, v):
    ...
    if v < self.value:
      self.left.insert(v)
      self.left.re_balance()
      self.height = 1 + max(self.left.height, self.right.height)
      return
    
    if v > self.value:
      self.right.insert(v)
      self.right.re_balance()
      self.height = 1 + max(self.left.height, self.right.height)
      return

### Summary

* Using rotations, we can maintain height balance
* Height balanced trees have height $O(log \ n)$
* `find(), insert()` and `delete()`all walk down a single path, take time $O(log \ n)$